In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [4]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()
SYSTEM_PROMPT_1

'Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.'

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_2_Food_Beverage_Production/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 17:54:19.645623


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Time periods: T = 12
- Products: P = {food 1, food 2, beverage 1, beverage 2, beverage 3}
- Fixed costs: FC_p,t (from "fixed_cost_production.csv")
- Variable costs: VC_p,t (from "variable_cost_production.csv")
- Variable storage costs: VC_s,p,t (from "variable_cost_storage.csv")
- Demand: D_p,t (from "demand.csv")
- Revenue: R_p,t (from "revenue.csv")
- Storage capacity: SC_p
- Production capacity: PC_p

Decision Variables:
- Production quantity: x_p,t (integer for food, real for beverage)
- Storage quantity: s_p,t (real)

Objective Function:
The objective is to maximize profit over the 12 time periods. The profit is calculated as revenue minus fixed and variable production costs and variable storage costs. The objective function is:

Maximize: Σ (R_p,t * x_p,t - FC_p,t - VC_p,t * x_p,t - VC_s,p,t * s_p,t)

Constraints:
The constraints for this problem are as follows:

1. Demand Constraint: The production quantity for each product in each time period cannot exceed the demand for that period.
   For all t in T and p in P: x_p,t ≤ D_p,t

2. Production Capacity Constraint: The production quantity for each product in each time period cannot exceed the production capacity for that period.
   For all t in T and p in P: x_p,t ≤ PC_p

3. Storage Capacity Constraint: The storage quantity for each product at the end of each time period cannot exceed the storage capacity for that product.
   For all t in T and p in P: s_p,t ≤ SC_p

4. Storage Balance Constraint: The storage quantity for each product at the end of each time period is equal to the storage quantity from the previous period, plus the production quantity for that period, minus the demand met in that period.
   For all t in T (except t = 1) and p in P: s_p,t = s_p,t-1 + x_p,t - min(x_p,t, D_p,t)

5. Non-negativity Constraints: The decision variables x_p,t and s_p,t must be non-negative.
   For all t in T and p in P: x_p,t ≥ 0, s_p,t ≥ 0

This optimization model can be solved using linear programming techniques to find the production and storage quantities that maximize profit over the 12 time periods.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:
- Time periods: T = 12
- Products: P = {food 1, food 2, beverage 1, beverage 2, beverage 3}
- Fixed costs: FC_p,t (from "fixed_cost_production.csv")
- Variable costs: VC_p,t (from "variable_cost_production.csv")
- Variable storage costs: VC_s,p,t (from "variable_cost_storage.csv")
- Demand: D_p,t (from "demand.csv")
- Revenue: R_p,t (from "revenue.csv")
- Storage capacity: SC_p
- Production capacity: PC_p

Decision Variables:
- Production quantity: x_p,t (integer for food, real for beverage)
- Storage quantity: s_p,t (real)

Objective Function:
The objective is to maximize profit over the 12 time periods. The profit is calculated as revenue minus fixed and variable production costs and variable storage costs. The objective function is:

Maximize: Σ (R_p,t * x_p,t - FC_p,t - VC_p,t * x_p,t - VC_s,p,t * s_p,t)

Co

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here's a Python Pyomo code for the optimization problem you described, using sample data for the parameters. I've indicated where I use sample data.

Please make sure to install the required packages by running `!pip install pyomo pandas` if you haven't already.

```python
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data)
T = 12
P = ['food 1', 'food 2', 'beverage 1', 'beverage 2', 'beverage 3']

# Load sample data from CSV files (replace with actual files)
fixed_cost_production = pd.read_csv('fixed_cost_production.csv', index_col=[0, 1])
variable_cost_production = pd.read_csv('variable_cost_production.csv', index_col=[0, 1])
variable_cost_storage = pd.read_csv('variable_cost_storage.csv', index_col=[0, 1])
demand = pd.read_csv('demand.csv', index_col=[0, 1])
revenue = pd.read_csv('revenue.csv', index_col=[0, 1])

# Storage and production capacity (replace with actual values)
storage_capacity = {p: 1000 for p in P}
production_capacity = {p: 500 for p in P}

# Create Pyomo model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, T + 1))
model.P = pyo.Set(initialize=P)

# Decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.s = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return sum(revenue.loc[(p, t), 'value'] * model.x[p, t] -
               fixed_cost_production.loc[(p, t), 'value'] -
               variable_cost_production.loc[(p, t), 'value'] * model.x[p, t] -
               variable_cost_storage.loc[(p, t), 'value'] * model.s[p, t]
               for p in model.P for t in model.T)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def demand_constraint_rule(model, p, t):
    return model.x[p, t] <= demand.loc[(p, t), 'value']

model.demand_constraint = pyo.Constraint(model.P, model.T, rule=demand_constraint_rule)

def production_capacity_constraint_rule(model, p, t):
    return model.x[p, t] <= production_capacity[p]

model.production_capacity_constraint = pyo.Constraint(model.P, model.T, rule=production_capacity_constraint_rule)

def storage_capacity_constraint_rule(model, p, t):
    return model.s[p, t] <= storage_capacity[p]

model.storage_capacity_constraint = pyo.Constraint(model.P, model.T, rule=storage_capacity_constraint_rule)

def storage_balance_constraint_rule(model, p, t):
    if t > 1:
        return model.s[p, t] == model.s[p, t - 1] + model.x[p, t] - min(model.x[p, t], demand.loc[(p, t), 'value'])
    else:
        return model.s[p, t] == model.x[p, t] - min(model.x[p, t], demand.loc[(p, t), 'value'])

model.storage_balance_constraint = pyo.Constraint(model.P, model.T, rule=storage_balance_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print solution
for p in model.P:
    for t in model.T:
        print(f'x_{p}_{t}: {model.x[p, t].value}, s_{p}_{t}: {model.s[p, t].value}')
```

Make sure to replace the sample data with your actual data and adjust the storage and production capacities as needed. The code assumes that the CSV files have the following structure:

- "fixed\_cost\_production.csv": A CSV file with columns 'product' and 'time' for the index, and a column 'value' for the fixed cost.
- "variable\_cost\_production.csv": A CSV file with columns 'product' and 'time' for the index, and a column 'value' for the variable cost.
- "variable\_cost\_storage.csv": A CSV file with columns 'product', 'time' for the index, and a column 'value' for the storage cost.
- "demand.csv": A CSV file with columns 'product' and 'time' for the index, and a column 'value' for the demand.
- "revenue.csv": A CSV file with columns 'product' and 'time' for the index, and a column 'value' for the revenue.

In [10]:
print(response_2_text)

Here's a Python Pyomo code for the optimization problem you described, using sample data for the parameters. I've indicated where I use sample data.

Please make sure to install the required packages by running `!pip install pyomo pandas` if you haven't already.

```python
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data)
T = 12
P = ['food 1', 'food 2', 'beverage 1', 'beverage 2', 'beverage 3']

# Load sample data from CSV files (replace with actual files)
fixed_cost_production = pd.read_csv('fixed_cost_production.csv', index_col=[0, 1])
variable_cost_production = pd.read_csv('variable_cost_production.csv', index_col=[0, 1])
variable_cost_storage = pd.read_csv('variable_cost_storage.csv', index_col=[0, 1])
demand = pd.read_csv('demand.csv', index_col=[0, 1])
revenue = pd.read_csv('revenue.csv', index_col=[0, 1])

# Storage and production capacity (replace with actual values)
storage_capacity = {p: 1000 for p in P}
production_capacity = {p: 500 fo

### Code Executability

In [12]:
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data)
T = 12
P = ['food 1', 'food 2', 'beverage 1', 'beverage 2', 'beverage 3']

# Load sample data from CSV files (replace with actual files)
fixed_cost_production = pd.read_csv(FILE_PATH_PROBLEM + 'fixed_cost_production.csv', index_col=[0, 1])
variable_cost_production = pd.read_csv(FILE_PATH_PROBLEM + 'variable_cost_production.csv', index_col=[0, 1])
variable_cost_storage = pd.read_csv(FILE_PATH_PROBLEM + 'variable_cost_storage.csv', index_col=[0, 1])
demand = pd.read_csv(FILE_PATH_PROBLEM + 'demand.csv', index_col=[0, 1])
revenue = pd.read_csv(FILE_PATH_PROBLEM + 'revenue.csv', index_col=[0, 1])

# Storage and production capacity (replace with actual values)
storage_capacity = {p: 1000 for p in P}
production_capacity = {p: 500 for p in P}

# Create Pyomo model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=range(1, T + 1))
model.P = pyo.Set(initialize=P)

# Decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.s = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return sum(revenue.loc[(p, t), 'value'] * model.x[p, t] -
               fixed_cost_production.loc[(p, t), 'value'] -
               variable_cost_production.loc[(p, t), 'value'] * model.x[p, t] -
               variable_cost_storage.loc[(p, t), 'value'] * model.s[p, t]
               for p in model.P for t in model.T)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def demand_constraint_rule(model, p, t):
    return model.x[p, t] <= demand.loc[(p, t), 'value']

model.demand_constraint = pyo.Constraint(model.P, model.T, rule=demand_constraint_rule)

def production_capacity_constraint_rule(model, p, t):
    return model.x[p, t] <= production_capacity[p]

model.production_capacity_constraint = pyo.Constraint(model.P, model.T, rule=production_capacity_constraint_rule)

def storage_capacity_constraint_rule(model, p, t):
    return model.s[p, t] <= storage_capacity[p]

model.storage_capacity_constraint = pyo.Constraint(model.P, model.T, rule=storage_capacity_constraint_rule)

def storage_balance_constraint_rule(model, p, t):
    if t > 1:
        return model.s[p, t] == model.s[p, t - 1] + model.x[p, t] - min(model.x[p, t], demand.loc[(p, t), 'value'])
    else:
        return model.s[p, t] == model.x[p, t] - min(model.x[p, t], demand.loc[(p, t), 'value'])

model.storage_balance_constraint = pyo.Constraint(model.P, model.T, rule=storage_balance_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print solution
for p in model.P:
    for t in model.T:
        print(f'x_{p}_{t}: {model.x[p, t].value}, s_{p}_{t}: {model.s[p, t].value}')

2024-05-22 18:08:59,527 ERROR pyomo.core: Rule failed when generating expression for Objective obj with index None:
KeyError: 'value'
2024-05-22 18:08:59,528 ERROR pyomo.core: Constructing component 'obj' from data=None failed:
KeyError: 'value'


KeyError: 'value'

### Solution Correctness

In [25]:
import pyomo.environ as pyo
import pandas as pd

# Sample data (replace with actual data)
T = 12
P = ['Food 1', 'Food 2', 'Beverage 1', 'Beverage 2', 'Beverage 3']

# Load sample data from CSV files (replace with actual files)
fixed_cost_production = pd.read_csv(FILE_PATH_PROBLEM + 'fixed_cost_production.csv', index_col=[0])
variable_cost_production = pd.read_csv(FILE_PATH_PROBLEM + 'variable_cost_production.csv', index_col=[0])
variable_cost_storage = pd.read_csv(FILE_PATH_PROBLEM + 'variable_cost_storage.csv', index_col=[0])
demand = pd.read_csv(FILE_PATH_PROBLEM + 'demand.csv', index_col=[0])
revenue = pd.read_csv(FILE_PATH_PROBLEM + 'revenue.csv', index_col=[0])

# Storage and production capacity (replace with actual values)
storage_capacity = {p: 1000 for p in P}
production_capacity = {p: 500 for p in P}

# Create Pyomo model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.Set(initialize=[t for t in range(1, T + 1)])
model.P = pyo.Set(initialize=P)

# Decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.s = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return sum(revenue.loc[p][t - 1] * model.x[p, t] -
               fixed_cost_production.loc[p][t - 1] -
               variable_cost_production.loc[p][t - 1] * model.x[p, t] -
               variable_cost_storage.loc[p][t - 1] * model.s[p, t]
               for p in model.P for t in model.T)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def demand_constraint_rule(model, p, t):
    return model.x[p, t] <= demand.loc[p][t - 1]

model.demand_constraint = pyo.Constraint(model.P, model.T, rule=demand_constraint_rule)

def production_capacity_constraint_rule(model, p, t):
    return model.x[p, t] <= production_capacity[p]

model.production_capacity_constraint = pyo.Constraint(model.P, model.T, rule=production_capacity_constraint_rule)

def storage_capacity_constraint_rule(model, p, t):
    return model.s[p, t] <= storage_capacity[p]

model.storage_capacity_constraint = pyo.Constraint(model.P, model.T, rule=storage_capacity_constraint_rule)

def storage_balance_constraint_rule(model, p, t):
    if t > 1:
        return model.s[p, t] == model.s[p, t - 1] + model.x[p, t] - min(model.x[p, t], demand.loc[p][t - 1])
    else:
        return model.s[p, t] == model.x[p, t] - min(model.x[p, t], demand.loc[p][t - 1])

model.storage_balance_constraint = pyo.Constraint(model.P, model.T, rule=storage_balance_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print solution
for p in model.P:
    for t in model.T:
        print(f'x_{p}_{t}: {model.x[p, t].value}, s_{p}_{t}: {model.s[p, t].value}')

2024-05-22 18:17:45,577 ERROR pyomo.core: Rule failed when generating expression for Constraint storage_balance_constraint with index ('Food 1', 1):
PyomoException: Cannot convert non-constant Pyomo expression (4.61  <  x[Food 1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
2024-05-22 18:17:45,578 ERROR pyomo.core: Constructing component 'storage_balance_constraint' from data=None failed:
PyomoException: Cannot convert non-constant Pyomo expression (4.61  <  x[Food 1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>

PyomoException: Cannot convert non-constant Pyomo expression (4.61  <  x[Food 1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.